In [9]:
# JUPYTER EFOM RUNNER — CONFIG

from pathlib import Path
import pandas as pd
import numpy as np

# Core orchestrator + helpers
from src import main
from src.main import ensure_pi_download

# Data loading primitives (prioritize downloaded_csv)
from src.data_loading import DataPaths, ResampleConfig, DataPipeline

# SRTO / SPYRO plumbing
from src.srto_pipeline import SRTOConfig, RCOTSweepConfig, FeedConfig, SRTOPipeline
from src.srto_components import component_index, MW

# PI downloader types
from src.pipeline import DownloadConfig, AuthenticationMode

# ==== User knobs ====
MODE           = "online"            # 'historical' | 'closed_loop' | 'online'
DOWNLOAD_PI    = True                # pull from PI first (incremental)
OUT_DIR        = Path("prod_out/jupyter_v3")
INPUT_DIR      = Path("input")
INTER_DIR      = Path("intermediate")
DOWNLOADED_CSV = "pi_firstrow.csv"   # relative to INTER_DIR

# if you want historical window for non-online tests:
START_STR      = "2024-09-10"
END_STR        = "2025-09-22"        # None for open end

# SRTO bits (adjust paths if needed)
SRTO_DLL = Path(r"C:\Program Files\Pyrotec\SRTO")
SPY7S = [
    r"01. GF_HYBRID MODE_SRTO7_NAPH.SPY7",
    r"04. LF_NAPH MODE_SRTO7.SPY7",
    r"07. GF_GAS MODE_SRTO7.SPY7",
]

# Prices cleanup rule
REPLACE_PRICE_ZEROS = True

# Set output base for artifacts
main.set_out_dir_base(OUT_DIR)


In [10]:
pi_csv = INTER_DIR / DOWNLOADED_CSV
pi_csv.parent.mkdir(parents=True, exist_ok=True)


In [11]:
END_STR

'2025-09-22'

In [12]:
# ENSURE PI CSV (incremental if DOWNLOAD_PI or file missing)

pi_csv = INTER_DIR / DOWNLOADED_CSV
pi_csv.parent.mkdir(parents=True, exist_ok=True)

if DOWNLOAD_PI or not pi_csv.exists():
    cfg = DownloadConfig(
        pi_server="172.17.21.117",
        auth_mode=AuthenticationMode.WINDOWS_AUTHENTICATION,
        pi_username="", pi_password="", pi_domain=None,
        tz="Asia/Seoul",
        start_date=START_STR,
        end_date=(END_STR or START_STR),
        interval="1m",
        chunk_days=7,
        recorded=False,
        sheet_name="python_import",
        column_name="tags",
        input_dir=str(INPUT_DIR),
        tags_excel=(INPUT_DIR / "EFOM_input_data_tag_list.xlsx"),
        out_csv=pi_csv,
        out_parquet="",
        incremental=True,
    )
    _ = ensure_pi_download(cfg)   # writes/updates pi_firstrow.csv


[CONFIG] start_date=2024-09-01 00:00:00, end_date=NOW (override), interval=1m, recording=False, incremental=True
[CONFIG] end_date overridden → 2025-09-22 18:20 (Asia/Seoul)


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [13]:
probe_cols = [c for c in ["Gas Feed_chamber4","Gas Feed_chamber5","Gas Feed_chamber6"] if c in X_12h.columns]
print("[CHK] gas feed @latest:", {c: X_12h.at[latest, c] if c in X_12h.index.names or c in X_12h.columns else None for c in probe_cols})


NameError: name 'latest' is not defined

In [14]:
# LOAD DATA VIA DataPipeline (prioritizes downloaded_csv)
# from importlib import reload
# reload(src.data_loading)
paths = DataPaths(
    input_dir=INPUT_DIR,
    inter_dir=INTER_DIR,
    downloaded_csv=DOWNLOADED_CSV,           # << prioritize PI CSV
    input_excel="EFOM_input_data_tag_list.xlsx",

    # legacy files still used by pipeline for other tables
    prod_excel="1. 생산량 Data_'23.07~'25.05_R1_송부용.xlsx",
    furn_excel="2. Furnace Data_'23.07~'25.05_R0.xlsx",
    nap_excel="Nap Feed 조성분석값.xlsx",
    gas_excel="Gas Feed 조성분석값.xlsx",
    recycle_excel="6. 에탄 및 프로판 데이터.xlsx",
    price_csv="price.csv",
    util_excel="#1ECU 유틸리티사용량일별데이터.xlsx",
    fresh_excel="7. Gas Furnace Feed Data_'23.07~'25.05_r2.xlsx",

    # optional caches/headers as per your repo defaults
    prod_pkl="df_production_v4.pkl",
    furn_pkl="furnace.pkl",
    nap_pkl="df_feed_naptha.pkl",
    gas_pkl="df_feed_gas.pkl",
    fresh_pkl="df_feed_fresh_v3.pkl",
    rec_pkl="df_recycle.pkl",
    prod_header=2, furn_header=2, nap_header=1, gas_header=1, rec_header=4, fresh_header=3,
)

cfg = ResampleConfig(hour_freq='h', win12_freq='12h', win12_offset='9h')

feature_rename = {
    'Naph': 'Naphtha_chamber1', 'T-DAO': 'T-DAO_chamber1', 'DS': 'DS_chamber1',
    'RCOT Ave.': 'RCOT_chamber1', 'Excess O2': "Excess O2_chamber1",
    'Naph.1': 'Naphtha_chamber2', 'T-DAO.1': 'T-DAO_chamber2','DS.1': 'DS_chamber2',
    'RCOT Ave..1': 'RCOT_chamber2', 'Excess O2.1': "Excess O2_chamber2",
    'Naph.2': 'Naphtha_chamber3', 'T-DAO.2': 'T-DAO_chamber3','DS.2': 'DS_chamber3',
    'RCOT Ave..2': 'RCOT_chamber3', 'Excess O2.2': "Excess O2_chamber3",
    'Naph.3': 'Naphtha_chamber4', 'GAS': 'Gas Feed_chamber4','DS.3': 'DS_chamber4',
    'RCOT Ave..3': 'RCOT_chamber4', 'Excess O2.3': "Excess O2_chamber4",
    'Naph.4': 'Naphtha_chamber5', 'GAS.1': 'Gas Feed_chamber5','DS.4': 'DS_chamber5',
    'RCOT Ave..4': 'RCOT_chamber5', 'Excess O2.4': "Excess O2_chamber5",
    'Naph.5': 'Naphtha_chamber6', 'GAS.2': 'Gas Feed_chamber6','DS.5': 'DS_chamber6',
    'RCOT Ave..5': 'RCOT_chamber6', 'Excess O2.5': "Excess O2_chamber6",
}
target_rename  = { 'Unnamed: 36':'steam','ECU F/G':'fuel_gas','ECU Elec..1':'electricity' }

dp = DataPipeline(paths, cfg).run(feature_rename, target_rename)
art = dp.artifacts()

X_12h = art['X_12h']
Y_12h = art['Y_12h']
prices_df = art['price_df']

if REPLACE_PRICE_ZEROS:
    prices_df = prices_df.replace(0, pd.NA).ffill()
prices_df = prices_df[~prices_df.index.duplicated(keep='first')]

print("Loaded:", X_12h.shape, Y_12h.shape, prices_df.shape)


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_16424\816477952.py:56: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prices_df = prices_df.replace(0, pd.NA).ffill()


Loaded: (774, 98) (774, 9) (387, 13)


In [16]:
probe_cols = [c for c in ["Gas Feed_chamber4","Gas Feed_chamber5","Gas Feed_chamber6"] if c in X_12h.columns]
# print("[CHK] gas feed @latest:", {c: X_12h.at[latest, c] if c in X_12h.index.names or c in X_12h.columns else None for c in probe_cols})


In [18]:
X_12h

,Ethylene_prod,Propylene_prod,MixedC4_prod,RPG_prod,PFO_prod,C2Recycle,C3Recycle,Hydrogen_prod,Tail_Gas_prod,Ethane_prod,...,i-Butane_gas,RCOT_naphtha_chamber4,RCOT_gas_chamber4,RCOT_naphtha_chamber5,RCOT_gas_chamber5,RCOT_naphtha_chamber6,RCOT_gas_chamber6,FreshFeed_C3 LPG,FreshFeed_MX Offgas,feed_qty
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-01 09:00:00,66.961969,31.875999,18.915362,33.082066,5.525592,17.748620,10.451746,1.932102,33.937773,17.748620,...,1.921667,880.027119,880.020830,0.0,28.965482,0.0,880.026279,45.032904,15.960949,60.993853
2024-09-01 21:00:00,67.015402,31.322504,19.002719,32.443917,5.491209,17.645969,10.903391,1.930902,34.434181,17.645969,...,1.921667,879.992959,879.987157,0.0,28.254550,0.0,879.991018,45.212228,15.932669,61.144897
2024-09-02 09:00:00,69.300842,32.614699,19.870455,34.842525,5.197984,18.398275,10.941583,1.971435,35.374823,18.398275,...,1.973750,879.442822,879.372944,0.0,27.293488,0.0,879.849443,43.909099,15.738533,59.647631
2024-09-02 21:00:00,74.605789,35.845847,21.142220,37.968441,4.312466,19.974149,10.643060,2.053634,38.222157,19.974149,...,1.973750,880.012074,879.971904,0.0,95.850268,0.0,879.981582,41.988303,15.891017,57.879320
2024-09-03 09:00:00,74.808790,35.416598,21.110860,36.018180,6.923870,20.133305,9.999946,2.052681,38.259773,20.133305,...,1.915000,879.985571,879.980632,0.0,669.419350,0.0,879.978009,42.274970,15.691889,57.966860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-20 21:00:00,80.189162,41.233319,22.422373,40.293263,9.210038,19.499534,15.651687,2.305761,39.389699,19.499534,...,1.589583,0.000000,885.016104,0.0,884.983174,0.0,0.000000,60.941494,17.859549,78.801043
2025-09-21 09:00:00,79.254790,41.400782,22.642964,44.251763,8.512621,18.254823,16.376972,2.242943,39.861093,18.254823,...,1.582500,0.000000,884.968799,0.0,884.957212,0.0,0.000000,64.448326,15.381159,79.829485
2025-09-21 21:00:00,80.346128,40.415717,22.585680,39.392188,8.033801,18.368733,16.985584,2.178664,39.696486,18.368733,...,1.582500,0.000000,884.933871,0.0,884.897974,0.0,0.000000,63.201942,15.068631,78.270573


In [22]:
X_12h

,Ethylene_prod,Propylene_prod,MixedC4_prod,RPG_prod,PFO_prod,C2Recycle,C3Recycle,Hydrogen_prod,Tail_Gas_prod,Ethane_prod,...,i-Butane_gas,RCOT_naphtha_chamber4,RCOT_gas_chamber4,RCOT_naphtha_chamber5,RCOT_gas_chamber5,RCOT_naphtha_chamber6,RCOT_gas_chamber6,FreshFeed_C3 LPG,FreshFeed_MX Offgas,feed_qty
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-01 09:00:00,66.961969,31.875999,18.915362,33.082066,5.525592,17.748620,10.451746,1.932102,33.937773,17.748620,...,1.921667,880.027119,880.020830,0.0,28.965482,0.0,880.026279,45.032904,15.960949,60.993853
2024-09-01 21:00:00,67.015402,31.322504,19.002719,32.443917,5.491209,17.645969,10.903391,1.930902,34.434181,17.645969,...,1.921667,879.992959,879.987157,0.0,28.254550,0.0,879.991018,45.212228,15.932669,61.144897
2024-09-02 09:00:00,69.300842,32.614699,19.870455,34.842525,5.197984,18.398275,10.941583,1.971435,35.374823,18.398275,...,1.973750,879.442822,879.372944,0.0,27.293488,0.0,879.849443,43.909099,15.738533,59.647631
2024-09-02 21:00:00,74.605789,35.845847,21.142220,37.968441,4.312466,19.974149,10.643060,2.053634,38.222157,19.974149,...,1.973750,880.012074,879.971904,0.0,95.850268,0.0,879.981582,41.988303,15.891017,57.879320
2024-09-03 09:00:00,74.808790,35.416598,21.110860,36.018180,6.923870,20.133305,9.999946,2.052681,38.259773,20.133305,...,1.915000,879.985571,879.980632,0.0,669.419350,0.0,879.978009,42.274970,15.691889,57.966860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-20 21:00:00,80.189162,41.233319,22.422373,40.293263,9.210038,19.499534,15.651687,2.305761,39.389699,19.499534,...,1.589583,0.000000,885.016104,0.0,884.983174,0.0,0.000000,60.941494,17.859549,78.801043
2025-09-21 09:00:00,79.254790,41.400782,22.642964,44.251763,8.512621,18.254823,16.376972,2.242943,39.861093,18.254823,...,1.582500,0.000000,884.968799,0.0,884.957212,0.0,0.000000,64.448326,15.381159,79.829485
2025-09-21 21:00:00,80.346128,40.415717,22.585680,39.392188,8.033801,18.368733,16.985584,2.178664,39.696486,18.368733,...,1.582500,0.000000,884.933871,0.0,884.897974,0.0,0.000000,63.201942,15.068631,78.270573


In [21]:
# after you build X_12h, before run_production
all_knobs = [c for c in X_12h.columns if c.startswith("RCOT")]
print("[DBG] knobs seen:", all_knobs)
missing = [k for k in ["RCOT_gas_chamber4","RCOT_gas_chamber5","RCOT_gas_chamber6"] if k not in all_knobs]
print("[DBG] missing gas knobs:", missing)


[DBG] knobs seen: ['RCOT_chamber1', 'RCOT_chamber2', 'RCOT_chamber3', 'RCOT #1_naphtha_chamber4', 'RCOT #3_naphtha_chamber4', 'RCOT #5_naphtha_chamber4', 'RCOT #7_naphtha_chamber4', 'RCOT #2_gas_chamber4', 'RCOT #4_gas_chamber4', 'RCOT #6_gas_chamber4', 'RCOT #8_gas_chamber4', 'RCOT #1_naphtha_chamber5', 'RCOT #3_naphtha_chamber5', 'RCOT #5_naphtha_chamber5', 'RCOT #7_naphtha_chamber5', 'RCOT #2_gas_chamber5', 'RCOT #4_gas_chamber5', 'RCOT #6_gas_chamber5', 'RCOT #8_gas_chamber5', 'RCOT #1_naphtha_chamber6', 'RCOT #3_naphtha_chamber6', 'RCOT #5_naphtha_chamber6', 'RCOT #7_naphtha_chamber6', 'RCOT #2_gas_chamber6', 'RCOT #4_gas_chamber6', 'RCOT #6_gas_chamber6', 'RCOT #8_gas_chamber6', 'RCOT_naphtha_chamber4', 'RCOT_gas_chamber4', 'RCOT_naphtha_chamber5', 'RCOT_gas_chamber5', 'RCOT_naphtha_chamber6', 'RCOT_gas_chamber6']
[DBG] missing gas knobs: []


In [20]:
print("[CHK] gas feed @latest:", {c: X_12h.at['2025-09-22 21:00:00', c] if c in X_12h.index.names or c in X_12h.columns else None for c in probe_cols})


[CHK] gas feed @latest: {'Gas Feed_chamber4': np.float64(58.845905840778144), 'Gas Feed_chamber5': np.float64(58.968266346815554), 'Gas Feed_chamber6': np.float64(0.0)}


In [27]:
latest = '2025-09-22 21:00:00'

In [29]:
bounds

(['RCOT_chamber1', 'RCOT_chamber3', 'RCOT_gas_chamber4', 'RCOT_gas_chamber5'],
 [(810.0, 853.0), (810.0, 853.0), (850.0, 890.0), (850.0, 890.0)])

In [28]:
try:
    from src.optimizer import _active_rcot_vars  # or wherever you define bounds
    bounds = _active_rcot_vars(X_12h.loc[latest])
    print("[DBG] bounds:", {k: bounds.get(k) for k in ["RCOT_gas_chamber4","RCOT_gas_chamber5","RCOT_gas_chamber6"]})
except Exception as e:
    print("[DBG] can't fetch bounds:", e)


[DBG] can't fetch bounds: 'tuple' object has no attribute 'get'


In [9]:
probe_cols = [c for c in ["Gas Feed_chamber4","Gas Feed_chamber5","Gas Feed_chamber6"] if c in X_12h.columns]
print("[CHK] gas feed @latest:", {c: X_12h.at[latest, c] if c in X_12h.index.names or c in X_12h.columns else None for c in probe_cols})


NameError: name 'latest' is not defined

In [ ]:
import importlib, src.main as m
importlib.reload(m)

pi_df = pd.read_csv(pi_csv)
pi_df['timestamp'] = pd.to_datetime(pi_df['timestamp'], errors='coerce')
density_df = pd.read_excel("./input/density_table.xlsx")

merged_lims = m.build_merged_lims_full(
    pi_df,
    X_12h,
    density_df=density_df,                 # raw; normalizer handles Group/CNR/Components Density 15
    bulk_density_col="M10L41004_Density",
    tolerance_days=7,
    enforce_100=False
)

# columns now include:
# - every "M10L41004_C* <Family>(wt%)"
# - family total "(wt%)" columns
# - tidy PONA columns: ['Paraffins','Olefins','Naphthenes','Aromatics']
# - gas columns from X_12h
print(merged_lims.columns.tolist()[:20], " ...")
merged_lims.tail()


['date', 'M10L41004_C4 n-Paraffin(wt%)', 'M10L41004_C5 n-Paraffin(wt%)', 'M10L41004_C6 n-Paraffin(wt%)', 'M10L41004_C7 n-Paraffin(wt%)', 'M10L41004_C8 n-Paraffin(wt%)', 'M10L41004_C9 n-Paraffin(wt%)', 'M10L41004_C10 n-Paraffin(wt%)', 'M10L41004_C11+ n-Paraffin(wt%)', 'M10L41004_C4 i-Paraffin(wt%)', 'M10L41004_C5 i-Paraffin(wt%)', 'M10L41004_C6 i-Paraffin(wt%)', 'M10L41004_C7 i-Paraffin(wt%)', 'M10L41004_C8 i-Paraffin(wt%)', 'M10L41004_C9 i-Paraffin(wt%)', 'M10L41004_C10 i-Paraffin(wt%)', 'M10L41004_C11+ i-Paraffin(wt%)', 'M10L41004_C4 Olefin(wt%)', 'M10L41004_C5 Olefin(wt%)', 'M10L41004_C6 Olefin(wt%)']  ...


,date,M10L41004_C4 n-Paraffin(wt%),M10L41004_C5 n-Paraffin(wt%),M10L41004_C6 n-Paraffin(wt%),M10L41004_C7 n-Paraffin(wt%),M10L41004_C8 n-Paraffin(wt%),M10L41004_C9 n-Paraffin(wt%),M10L41004_C10 n-Paraffin(wt%),M10L41004_C11+ n-Paraffin(wt%),M10L41004_C4 i-Paraffin(wt%),...,Ethane,Propylene,Propane,n-Butane,i-Butane,i-Butane_gas,Ethane_gas,Propylene_gas,Propane_gas,n-Butane_gas
385,2025-09-17 19:00:00,1.110692,25.050528,13.967787,2.751839,0.079139,0.034616,0.02354,0.0,0.044075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,2025-09-18 07:00:00,0.547586,22.319245,15.191798,2.931328,0.022295,0.022755,0.00000,0.0,0.017384,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,2025-09-18 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,2025-09-19 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,2025-09-19 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import re

renames = {}
to_drop = []

for c in merged_lims.columns:
    # 1) Per-carbon: M10L41004_C<NN or NN+> <Family>(wt%) → C<NN> <Family>
    m = re.match(r'^M10L41004_C(?P<C>\d+\+?)\s+(?P<FAM>[^()]+)\(wt%\)$', c)
    if m:
        renames[c] = f"C{m.group('C')} {m.group('FAM')}"
        continue

    # 2) Family totals (prefixed): drop the ones that duplicate tidy PONA,
    #    keep n-/i-Paraffin (rename without prefix/suffix)
    m2 = re.match(r'^M10L41004_(?P<FAM>Paraffins|Olefins|Naphthenes|Aromatics|n-Paraffin|i-Paraffin)\(wt%\)$', c)
    if m2:
        fam = m2.group('FAM')
        if fam in ('Paraffins','Olefins','Naphthenes','Aromatics'):
            # already have tidy columns with these names → drop the prefixed duplicates
            to_drop.append(c)
        else:
            # keep n-/i-Paraffin totals, but clean the name
            renames[c] = fam

# apply
merged_lims_clean = merged_lims.drop(columns=to_drop).rename(columns=renames)

# quick peek

merged_lims_clean.columns.tolist()[:25]
merged_lims = merged_lims_clean.copy()

In [ ]:

gas_cols_x = ['Ethane_gas','Propylene_gas','Propane_gas','n-Butane_gas','i-Butane_gas']
gas_cols   = ['Ethane','Propylene','Propane','n-Butane','i-Butane']
TOL        = pd.Timedelta(hours=3)   # 07:00<->09:00, 19:00<->21:00

# 1) drop any pre-existing gas cols from merged_lims
merged_lims = merged_lims.drop(columns=[c for c in gas_cols_x+gas_cols if c in merged_lims.columns],
                               errors='ignore').copy()

# 2) ensure we have a 'date' column (tz-naive) and it's sorted
if 'date' not in merged_lims.columns:
    merged_lims = merged_lims.reset_index().rename(columns={'index':'date'})
merged_lims['date'] = pd.to_datetime(merged_lims['date'], errors='coerce')
merged_lims = merged_lims.sort_values('date').reset_index(drop=True)

targets = merged_lims[['date']].copy()

# 3) pull only the *_gas columns that exist
gas_cols_x = ['Ethylene_gas','Ethane_gas','Propylene_gas','Propane_gas','n-Butane_gas','i-Butane_gas']
have = [c for c in gas_cols_x if c in X_12h.columns]

if have:
    g = (X_12h[have].sort_index()
                      .reset_index()
                      .rename(columns={X_12h.index.name or X_12h.columns[0]: 'ts'}))
    g['ts'] = pd.to_datetime(g['ts'], errors='coerce')

    # 4) nearest-asof (maps 07→09, 19→21 within ±3h)
    out = pd.merge_asof(
        left=targets.sort_values('date'),
        right=g.sort_values('ts'),
        left_on='date', right_on='ts',
        direction='nearest', tolerance=TOL
    ).drop(columns=['ts'])

    # 5) keep 'date' in attach and strip '_gas' → canonical names
    attach = out[['date'] + have].copy()
    attach = attach.rename(columns={c: c.replace('_gas','') for c in have})

    # 6) merge on 'date' (not by index), then ffill canonical cols
    merged_lims = (merged_lims
                   .merge(attach, on='date', how='left')
                   .sort_values('date')
                   .reset_index(drop=True))

    canon = [c.replace('_gas','') for c in have]
    merged_lims[canon] = merged_lims[canon].ffill()

    # (optional) also keep *_gas for legacy downstreams
    for c in canon:
        cg = c + '_gas'
        if cg not in merged_lims.columns:
            merged_lims[cg] = merged_lims[c]
else:
    # nothing to add; ensure columns exist as NaN to avoid KeyErrors later
    for c in gas_cols:
        if c not in merged_lims.columns:
            merged_lims[c] = pd.NA
    for c in gas_cols_x:
        if c not in merged_lims.columns:
            merged_lims[c] = pd.NA


In [ ]:
import pandas as pd
from typing import Literal, List

def unify_gas_columns(df: pd.DataFrame,
                      prefer: Literal["gas","canon"] = "gas",
                      keep_both: bool = False,
                      log_diff: bool = True,
                      atol: float = 1e-9) -> pd.DataFrame:
    canon = ["Ethylene","Ethane","Propylene","Propane","n-Butane","i-Butane"]
    out = df.copy()

    diffs: List[str] = []
    for c in canon:
        cg = f"{c}_gas"
        has_c  = c  in out.columns
        has_cg = cg in out.columns

        if not has_c and not has_cg:
            continue

        if prefer == "gas":
            # make/keep canonical column, prefer *_gas values, fill from canonical if missing
            if not has_c:
                out[c] = pd.NA
            if has_cg:
                out[c] = out[cg].where(out[cg].notna(), out[c])
        else:  # prefer == "canon"
            if has_c:
                pass  # canonical stays as is
            elif has_cg:
                out[c] = out[cg]  # create canonical from gas

        # optional diff log (only where both present and finite)
        if log_diff and has_c and has_cg:
            a = pd.to_numeric(out[c],  errors="coerce")
            b = pd.to_numeric(out[cg], errors="coerce")
            neq = (a.notna() & b.notna()) & ((a - b).abs() > atol)
            if neq.any():
                diffs.append(f"{c}: {neq.sum()} differing rows")

        # drop or keep *_gas
        if not keep_both and cg in out.columns:
            out = out.drop(columns=[cg])

    if log_diff and diffs:
        print("[unify_gas_columns] Differences detected ->\n  " + "\n  ".join(diffs))
    return out
# Prefer the *_gas series as the source of truth, fill gaps from canonical, and drop *_gas after:
merged_lims = unify_gas_columns(merged_lims, prefer="gas", keep_both=False)

# If you want to keep both for downstream compatibility:
# merged_lims = unify_gas_columns(merged_lims, prefer="gas", keep_both=True)


In [ ]:
# from typing import List

# def enforce_gas_suffix(df: pd.DataFrame,
#                        canon: List[str] = ["Ethylene","Ethane","Propylene","Propane","n-Butane","i-Butane"],
#                        keep_canon: bool = False) -> pd.DataFrame:
#     """
#     Prefer *_gas columns; if missing/NaN, backfill from canonical.
#     Create *_gas from canonical when *_gas doesn't exist.
#     Drop canonical columns unless keep_canon=True.
#     """
#     out = df.copy()
#     for c in canon:
#         cg = f"{c}_gas"
#         has_c  = c  in out.columns
#         has_cg = cg in out.columns

#         if has_cg and has_c:
#             # keep *_gas values, fill its NaNs from canonical
#             out[cg] = pd.to_numeric(out[cg], errors="coerce").where(out[cg].notna(), pd.to_numeric(out[c], errors="coerce"))
#         elif not has_cg and has_c:
#             # create *_gas entirely from canonical
#             out[cg] = pd.to_numeric(out[c], errors="coerce")
#         # else: only *_gas exists → nothing to do

#         if not keep_canon and has_c:
#             out = out.drop(columns=[c])

#     return out
# merged_lims = enforce_gas_suffix(merged_lims, keep_canon=False)


In [ ]:
# SRTO + Spyro memo

sel_spy7 = [(p if Path(p).is_absolute() else (SRTO_DLL / p)) for p in (SPY7S or [])]
srto_config  = SRTOConfig(SRTO_DLL, sel_spy7, component_index, MW)
sweep_config = RCOTSweepConfig(rcot_min=790.0, rcot_max=900.0, rcot_step=2.0,
                               chunk_size=10, n_jobs=6, save_checkpoints=True)

canonical_gas = ['Ethylene','Ethane','Propylene','Propane','n-Butane','i-Butane']
gas_cols = [c for c in canonical_gas if c in X_12h.columns] or canonical_gas
feed_config  = FeedConfig(gas_components=gas_cols)

pipeline = SRTOPipeline(srto_config, sweep_config, feed_config)

# Spyro memoized function
_SHORT_TO_SRTO = {
    'Ethylene':'Ethylene','Propylene':'Propylene','MixedC4':'MixedC4','RPG':'RPG',
    'Ethane':'Ethane','Propane':'Propane',
    'Fuel_Gas':'Fuel_Gas','Fuel Gas':'Fuel_Gas','FG':'Fuel_Gas','FuelGas':'Fuel_Gas',
    'Tail Gas':'Tail_Gas', 'Tail_Gas':'Tail_Gas'
}

class _SpyroMemo:
    def __init__(self, fn, key_cols=None, decimals=4, maxsize=200000):
        self.fn = fn; self.key_cols = tuple(key_cols) if key_cols is not None else None
        self.dec = decimals; self.cache = {}; self.maxsize = maxsize
    def _select_cols(self, row):
        if self.key_cols is None:
            return tuple(c for c in row.index if c.startswith('RCOT') or c.startswith('Naphtha_chamber') or c.startswith('Gas Feed_chamber'))
        return self.key_cols
    def _to_num(self, x):
        try: v = float(x)
        except Exception: v = 0.0
        if v != v: v = 0.0
        return round(v, self.dec)
    def _sig(self, row, short_key):
        cols = self._select_cols(row)
        vals = tuple(self._to_num(row.get(c, 0.0)) for c in cols)
        return (short_key, cols, vals)
    def __call__(self, row, short_key, ctx=None):
        k = self._sig(row, short_key); v = self.cache.get(k)
        if v is not None: return v
        out = self.fn(row, short_key, ctx)
        if len(self.cache) < self.maxsize: self.cache[k] = out
        return out

# def _make_spyro_fn(pipeline, merged_lims):
#     def _spyro_row(row_like: pd.Series, short_key: str, ctx=None) -> float:
#         ts = getattr(row_like, 'name', None)
#         if ts is None: return 0.0
#         sel = merged_lims.loc[merged_lims['date'] <= ts]
#         comp_row = (sel.iloc[-1] if not sel.empty else merged_lims.iloc[0])
#         spot = pipeline.predict_spot_plant(row_like, comp_row, feed_thr=0.1)
#         if spot.get('status') != 'ok': return 0.0
#         key = _SHORT_TO_SRTO.get(short_key, short_key)
#         return float(spot['totals_tph'].get(key, 0.0))
#     return _SpyroMemo(_spyro_row)

def _make_spyro_fn(pipeline, merged_lims):
    # merged_lims must have date index per step (1)
    def _spyro_row(row_like: pd.Series, short_key: str, ctx=None) -> float:
        ts = getattr(row_like, 'name', None)
        if ts is None:
            return 0.0
        # ts is tz-naive; merged_lims index is tz-naive → safe as-of selection
        # choose the last daily row <= ts
        try:
            comp_row = merged_lims.loc[:ts].iloc[-1]
        except Exception:
            comp_row = merged_lims.iloc[0]
        spot = pipeline.predict_spot_plant(row_like, comp_row, feed_thr=0.1)
        if spot.get('status') != 'ok':
            return 0.0
        key = _SHORT_TO_SRTO.get(short_key, short_key)
        return float(spot['totals_tph'].get(key, 0.0))
    return _SpyroMemo(_spyro_row)


spyro_fn = _make_spyro_fn(pipeline, merged_lims)
print("SRTO + Spyro ready. Gas cols:", gas_cols)


SRTO + Spyro ready. Gas cols: ['Ethylene', 'Ethane', 'Propylene', 'Propane', 'n-Butane', 'i-Butane']


In [ ]:
# # from X_12h.columns how do i drop the suffix '_gas'
# X_12h.columns = X_12h.columns.str.replace('_gas', '', regex=False)

In [ ]:
from src.data_loading import load_feed_data

merged_lims2 = load_feed_data(
    nap_path=paths.input_dir / "복사본 (2024-25) ECU 투입 납사 세부성상-wt%.xlsx",
    gas_path=paths.input_dir / "Gas Feed 조성분석값.xlsx", header=1
)
merged_lims2['date'] = pd.to_datetime(merged_lims2['date'], errors='coerce')
merged_lims2 = merged_lims2.dropna(subset=['date']).sort_values('date')

gas_cols = [c for c in ['Ethylene','Ethane','Propylene','Propane','n-Butane','i-Butane'] if c in merged_lims2.columns]
zr = (merged_lims2[gas_cols].sum(axis=1) == 0)
merged_lims2.loc[zr, gas_cols] = np.nan
merged_lims2[gas_cols] = merged_lims2[gas_cols].ffill().bfill()
merged_lims2 = merged_lims2.iloc[4:]  # keep (as you had)


c:\Users\Administrator\Documents\src\data_loading.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feed_gas['date'] = pd.to_datetime(feed_gas['date'], errors='coerce')


In [ ]:
# --- alias fixes for per-carbon names (source -> target) ---
alias_map = {
    'C11 n-Paraffin': 'C11+ n-Paraffin',
    'C11 i-Paraffin': 'C11+ i-Paraffin',
    # add more if you encounter them later, e.g.:
    # 'C11 Naphthene': 'C11+ Naphthene',
    # 'C11 Olefin'   : 'C11+ Olefin',
}

# Only create aliases when target name is missing and source name exists.
to_add = {}
for src, tgt in alias_map.items():
    if src in merged_lims2.columns and tgt not in merged_lims2.columns:
        to_add[tgt] = pd.to_numeric(merged_lims2[src], errors='coerce')

if to_add:
    merged_lims2 = merged_lims2.copy()
    for tgt, s in to_add.items():
        merged_lims2[tgt] = s


In [ ]:

# 0) standardize 'date' on both frames
def _ensure_date_col(df):
    if 'date' in df.columns:
        d = pd.to_datetime(df['date'], errors='coerce')
    elif isinstance(df.index, pd.DatetimeIndex):
        d = pd.to_datetime(df.index)
        df = df.reset_index(drop=True)
    else:
        raise ValueError("Need a datetime index or 'date' column")
    df = df.copy()
    df['date'] = d.dt.tz_convert('Asia/Seoul').dt.tz_localize(None) if getattr(d, 'dt', None) is not None and d.dt.tz is not None else d
    return df

merged_lims  = _ensure_date_col(merged_lims)
merged_lims2 = _ensure_date_col(merged_lims2)

merged_lims  = merged_lims.sort_values('date').reset_index(drop=True)
merged_lims2 = merged_lims2.sort_values('date').reset_index(drop=True)

# 1) define columns to import from merged_lims2
pona_fam = ['Paraffins','n-Paraffin','i-Paraffin','Olefins','Naphthenes','Aromatics']
gas_can  = ['Ethylene','Ethane','Propylene','Propane','n-Butane','i-Butane']

# any per-carbon like 'C6 Aromatic', 'C10 Olefin', etc.
pat = re.compile(r'^C(4|5|6|7|8|9|10|11|11\+|12\+)\s+(n-?Paraffin|i-?Paraffin|Olefin|Naphthene|Aromatic)$', re.I)
per_carbon = [c for c in merged_lims2.columns if pat.match(str(c))]

src_cols = [c for c in (pona_fam + gas_can + per_carbon) if c in merged_lims2.columns]

# 2) map merged_lims2 onto merged_lims timestamps (backward as-of within 1 day)
ml2_map = pd.merge_asof(
    left=merged_lims[['date']].sort_values('date'),
    right=merged_lims2[['date'] + src_cols].sort_values('date'),
    left_on='date', right_on='date',
    direction='backward', tolerance=pd.Timedelta(days=1)
)

# 3) only fill where merged_lims is missing; keep existing values
for c in src_cols:
    if c not in merged_lims.columns:
        merged_lims[c] = pd.NA
    merged_lims[c] = pd.to_numeric(merged_lims[c], errors='coerce')
    merged_lims[c] = merged_lims[c].where(merged_lims[c].notna(), pd.to_numeric(ml2_map[c], errors='coerce'))

# 4) short ffill/bfill to bridge ~2 weeks (07/19 sampling ≈ 28 stamps)
fill_cols = [c for c in (pona_fam + gas_can + per_carbon) if c in merged_lims.columns]
merged_lims[fill_cols] = (merged_lims[fill_cols]
                          .ffill(limit=60)
                          .bfill(limit=60))

# 5) (optional) if downstream wants *_gas too, mirror canonical → *_gas
for c in gas_can:
    cg = f'{c}_gas'
    if cg not in merged_lims.columns:
        merged_lims[cg] = merged_lims[c]

# 6) put 'date' back as index if you want
merged_lims = merged_lims.set_index('date').sort_index()


In [ ]:
merged_lims['date'] = merged_lims.index
merged_lims.reset_index(drop=True, inplace=True)

In [ ]:
# PICK WINDOW AND RUN

if MODE == 'online':
    latest = X_12h.index.max()
    start  = latest.normalize()
    end    = start
    online_opts = dict(online_latest_only=True)
else:
    start = pd.to_datetime(START_STR)
    end   = (pd.to_datetime(END_STR) if END_STR else None)
    online_opts = {}

act_hook = (main.default_actuation_logger_factory(OUT_DIR) if MODE=='online' else None)

main.run_production(
    X_12h=X_12h, Y_12h=Y_12h, merged_lims=merged_lims, pipeline=pipeline,
    prices_df=prices_df, total_spyro_yield_for_now=spyro_fn,
    start=start, end=end, mode=MODE,
    closed_loop_opts=dict(
        apply_timing='next_day',
        hold_policy='hold_until_next',
        ml_train_mode='historical',
        gp_train_mode='historical',
        cache_tag=('' if MODE=='historical' else '_sim'),
        **online_opts,
    ),
    act_hook=act_hook,
)


2025-09-18 09:00:00
{'Ethylene_prod_t+1': 82.2311773354749, 'Propylene_prod_t+1': 38.940684541487784, 'MixedC4_prod_t+1': 23.082061595601854, 'RPG_prod_t+1': 42.300182372107216, 'Ethane_prod_t+1': 20.98967302308598, 'Propane_prod_t+1': 14.714746309011208, 'Hydrogen_prod_t+1': 2.400816655382199, 'Tail_Gas_prod_t+1': 39.939026928450836}
fidelity_alpha_done
fidelity_passed
optimization started
optimization done

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 901.27
RCOT* (per chamber):
  RCOT_chamber1              844.99 →  849.99  (Δ +5.00)
  RCOT_chamber3              844.96 →  849.96  (Δ +5.00)
  RCOT_gas_chamber4          884.56 →  889.56  (Δ +5.00)
  RCOT_gas_chamber5          884.96 →  889.96  (Δ +5.00)

=== PRICE SNAPSHOT @ 2025-09-18 09:00:00 ===
{'Ethylene': 786.4918212890625, 'Propylene': 0.0, 'Mixed C4': 750.9017333984375, 'RPG': 611.431640625, 'Hydrogen': 1735.5777587890625, 'Tail Gas': 629.0755004882812, 'Fuel Gas': 587.0800170898438, 'PN': 618.875, 'Gas Feed': 0.0, 'LPG':

In [23]:
aa = pd.read_csv('./intermediate/pi_firstrow.csv')

In [ ]:

def _latest_recs_from_outdir(out_dir: Path):
    # try usual names, then wildcard
    cand = [
        out_dir /"online" / "rcot_recommendations_sim.csv",
        out_dir /"online" / "rcot_recommendations.csv",
    ]
    if not any(p.exists() for p in cand):
        # fallback to first matching file if any
        wild = sorted(out_dir.glob("rcot_recommendations*.csv"))
        if wild:
            cand.append(wild[-1])
    rec_path = next((p for p in cand if p.exists()), None)
    if not rec_path:
        return None, {}, {}, {}
    recs = pd.read_csv(rec_path, parse_dates=["timestamp"]).sort_values("timestamp")
    last = recs.iloc[-1]
    ts = pd.Timestamp(last["timestamp"])
    # rcots
    rcots = {}
    for c in recs.columns:
        if c.startswith("rcot_opt_") and pd.notna(last[c]):
            rcots[c.replace("rcot_opt_", "")] = float(last[c])
    # products (current baseline; switch to *_opt_tph if you want optimal)
    prods = {}
    for p in ["Ethylene","Propylene","MixedC4","RPG"]:
        col = f"{p}_opt_tph"
        if col in recs.columns and pd.notna(last[col]):
            prods[p] = float(last[col])
    # margin/performance if you stored them (else zeros)
    extras = dict(
        margin_hourly=float(last.get("margin_current_per_h", 0.0)) if pd.notna(last.get("margin_current_per_h", np.nan)) else 0.0,
        performance=0.0,  # plug your own score if you have it
    )
    return ts, rcots, prods, extras

In [ ]:

def _build_push_record(out_dir: Path, X_12h: pd.DataFrame, Y_12h: pd.DataFrame, mape: dict | None = None):
    ts, rcots, prods, extras = _latest_recs_from_outdir(out_dir)
    if ts is None:
        ts, rcots, prods, extras = _fallback_from_frames(X_12h, Y_12h)

    def pick(d, *keys):
        for k in keys:
            if k in d: return d[k]
        return None

    rec = {
        "rcot1":     pick(rcots, "RCOT_chamber1"),
        "rcot2":     pick(rcots, "RCOT_chamber2"),
        "rcot3":     pick(rcots, "RCOT_chamber3"),
        "rcot4_nap": pick(rcots, "RCOT_naphtha_chamber4"),
        "rcot4_gas": pick(rcots, "RCOT_gas_chamber4"),
        "rcot5_nap": pick(rcots, "RCOT_naphtha_chamber5"),
        "rcot5_gas": pick(rcots, "RCOT_gas_chamber5"),
        "rcot6_nap": pick(rcots, "RCOT_naphtha_chamber6"),
        "rcot6_gas": pick(rcots, "RCOT_gas_chamber6"),
        "eth_prod":  prods.get("Ethylene"),
        "prop_prod": prods.get("Propylene"),
        "mc4_prod":  prods.get("MixedC4"),
        "rpg_prod":  prods.get("RPG"),
        "margin_hourly": extras.get("margin_hourly", 0.0),
        "performance":   extras.get("performance", 0.0),
        "timestamp_str": pd.Timestamp(ts).strftime("%Y-%m-%d %H:%M:%S"),
    }
    if mape:
        rec.update({k: (None if v is None else float(v)) for k, v in mape.items()})

    rec = {k: v for k, v in rec.items() if v is not None}
    return ts, rec


# def _safe_publish_record(pub: PIPublisher, record: dict, ts):
#     tz = pub.cfg.tz
#     ts2 = pd.Timestamp(ts)
#     ts2 = ts2.tz_localize(tz) if ts2.tz is None else ts2.tz_convert(tz)
#     for key, tag in TAG_MAP.items():
#         if key not in record: continue
#         val = record[key]
#         if val is None or (isinstance(val, float) and np.isnan(val)): continue
#         try:
#             pt = pub._get_point(tag)
#             ptype = (getattr(pt, "pointtype", "") or "").lower()
#             v = str(val) if ptype == "string" else float(val)
#             pt.update_value(v, ts2.to_pydatetime(), UpdateMode.NO_REPLACE, BufferMode.BUFFER_IF_POSSIBLE)
#         except Exception as e:
#             print(f"[fail] {tag}: {e}")



In [ ]:
metrics = pd.read_excel('C:/Users/Administrator/Documents/prod_out/jupyter_v3/metrics.xlsx')

In [153]:
mape_for_push

{'mape_ethy': 1.587290104251739,
 'mape_prop': 2.220742953384054,
 'mape_mc4': 1.953615610380001,
 'mape_rpg': 3.701434194421058}

In [154]:
metrics

,target,n,rmse,mae,mape_pct,r2,bias,corr
0,Ethane_prod_t+1,181,0.919813,0.640821,2.817542,0.780572,-0.117676,0.885569
1,Ethylene_prod_t+1,181,3.003278,1.406353,1.587290,0.774192,-0.305109,0.881621
2,Hydrogen_prod_t+1,181,0.068656,0.042825,1.766119,0.703290,-0.007389,0.843065
3,MixedC4_prod_t+1,181,1.037474,0.507339,1.953616,0.850616,0.008403,0.922294
4,Propane_prod_t+1,181,0.894137,0.616743,4.715349,0.833602,-0.243648,0.919825
5,Propylene_prod_t+1,181,1.596635,0.931869,2.220743,0.753681,-0.247379,0.871852
6,RPG_prod_t+1,181,2.425014,1.610661,3.701434,0.842238,-0.098880,0.918231
7,Tail_Gas_prod_t+1,181,1.764118,0.955027,2.622183,0.796370,-0.082279,0.892840


In [155]:
def get(tgt, metrics):
    try:
        v = metrics.loc[tgt, 'mape_pct']
        return float(v) if pd.notna(v) else None
    except Exception:
        return None
aa = get('Ethylene_prod_t+1', metrics)

In [156]:
metrics[metrics['target'] == 'Ethylene_prod_t+1']['mape_pct'].mean()

np.float64(1.587290104251739)

In [157]:
aa

In [158]:

def _extract_mape_for_push(metrics: pd.DataFrame) -> dict:
    """Return {'mape_ethy': float|None, 'mape_prop': ..., 'mape_mc4': ..., 'mape_rpg': ...} from metrics df."""
    if metrics is None or metrics.empty:
        return {}
    def get(tgt):
        try:
            v =metrics[metrics['target'] == tgt]['mape_pct'].mean()
            return float(v) if pd.notna(v) else None
        except Exception:
            return None
    return {
        'mape_ethy': get('Ethylene_prod_t+1'),
        'mape_prop': get('Propylene_prod_t+1'),
        'mape_mc4':  get('MixedC4_prod_t+1'),
        'mape_rpg':  get('RPG_prod_t+1'),
    }

In [159]:
mape_for_push = _extract_mape_for_push(metrics)


In [160]:
mape_for_push

{'mape_ethy': 1.587290104251739,
 'mape_prop': 2.220742953384054,
 'mape_mc4': 1.953615610380001,
 'mape_rpg': 3.701434194421058}

In [5]:
OUT_DIR        = Path("prod_out/jupyter_v3")


In [6]:

def _fallback_from_frames(X_12h: pd.DataFrame, Y_12h: pd.DataFrame):
    last = X_12h.dropna(how="all").iloc[-1]
    ts = last.name
    rcots = {}
    for k in ["RCOT_chamber1","RCOT_chamber2","RCOT_chamber3",
              "RCOT_naphtha_chamber4","RCOT_gas_chamber4",
              "RCOT_naphtha_chamber5","RCOT_gas_chamber5",
              "RCOT_naphtha_chamber6","RCOT_gas_chamber6"]:
        if k in last and pd.notna(last[k]): rcots[k] = float(last[k])
    prods = {}
    if not Y_12h.empty and ts in Y_12h.index:
        for p in ["Ethylene","Propylene","MixedC4","RPG"]:
            col = f"{p}_prod_t+1"
            if col in Y_12h.columns and pd.notna(Y_12h.at[ts, col]):
                prods[p] = float(Y_12h.at[ts, col])
    extras = dict(margin_hourly=0.0, performance=0.0)
    return ts, rcots, prods, extras

In [7]:
ts_push

NameError: name 'ts_push' is not defined

In [8]:
ts_push, rec = _build_push_record(OUT_DIR, X_12h, Y_12h, mape=mape_for_push)


NameError: name '_build_push_record' is not defined

In [165]:
rec


{'rcot1': 844.8678791249838,
 'rcot3': 844.8229165601236,
 'rcot4_gas': 884.7312444051107,
 'rcot5_gas': 884.7832336425782,
 'eth_prod': 80.2329942262229,
 'prop_prod': 39.66314831375817,
 'mc4_prod': 21.572257048274647,
 'rpg_prod': 40.95550052808159,
 'margin_hourly': 50234.41828429593,
 'performance': 0.0,
 'timestamp_str': '2025-09-22 09:00:00',
 'mape_ethy': 1.587290104251739,
 'mape_prop': 2.220742953384054,
 'mape_mc4': 1.953615610380001,
 'mape_rpg': 3.701434194421058}

In [166]:
# QUICK ARTIFACT CHECKS (optional)

from pathlib import Path
import json

rcot_csv = (OUT_DIR / ('online' if MODE=='online' else MODE) / f"rcot_recommendations{'_sim' if MODE!='historical' else ''}.csv")
if rcot_csv.exists():
    df = pd.read_csv(rcot_csv, parse_dates=['timestamp'])
    display(df.tail(3))
else:
    print("rcot_recommendations not found:", rcot_csv)

aud_dir = OUT_DIR / ('online' if MODE=='online' else MODE) / "audits"
if aud_dir.exists():
    last_aud = sorted(aud_dir.glob("audit_summary_*.json"))[-1] if list(aud_dir.glob("audit_summary_*.json")) else None
    if last_aud:
        print("Last audit:", last_aud.name)
        print(json.loads(last_aud.read_text())['margins'] if 'margins' in json.loads(last_aud.read_text()) else "OK")


,timestamp,geometry,status,margin_baseline_per_h,margin_opt_per_h,improvement_per_h,margin_realized_per_h,margin_current_per_h,rcot_current_RCOT_chamber1,rcot_current_RCOT_chamber2,...,MixedC4_delta_tph,RPG_current_tph,RPG_opt_tph,RPG_delta_tph,Hydrogen_current_tph,Hydrogen_opt_tph,Hydrogen_delta_tph,Tail_Gas_current_tph,Tail_Gas_opt_tph,Tail_Gas_delta_tph
1,2025-09-19 21:00:00,GF_HYB_NAPH,ok,50986.368536,51267.791542,281.423006,NaN,50986.368536,840.601627,24.149854,...,-0.050755,39.903578,38.929413,-0.974164,2.365600,2.400369,0.034769,40.118087,40.580498,0.462411
2,2025-09-21 21:00:00,GF_HYB_NAPH,ok,48229.430439,48510.384082,280.953643,48057.229255,48229.430439,839.984756,22.996758,...,-0.046588,39.940834,38.938952,-1.001881,2.139223,2.174183,0.034960,39.636936,40.101936,0.465000
3,2025-09-22 09:00:00,GF_HYB_NAPH,ok,50234.418284,50563.184080,328.765796,NaN,50234.418284,839.867879,23.207926,...,-0.058396,41.952342,40.955501,-0.996841,2.299706,2.335481,0.035775,39.171301,39.659303,0.488002


Last audit: audit_summary_20250922_0900.json
OK


In [167]:
from src.pi_uploader import PIServerConfig, PIPublisher
from PIconnect.PIConsts import UpdateMode, BufferMode


In [168]:
rec

{'rcot1': 844.8678791249838,
 'rcot3': 844.8229165601236,
 'rcot4_gas': 884.7312444051107,
 'rcot5_gas': 884.7832336425782,
 'eth_prod': 80.2329942262229,
 'prop_prod': 39.66314831375817,
 'mc4_prod': 21.572257048274647,
 'rpg_prod': 40.95550052808159,
 'margin_hourly': 50234.41828429593,
 'performance': 0.0,
 'timestamp_str': '2025-09-22 09:00:00',
 'mape_ethy': 1.587290104251739,
 'mape_prop': 2.220742953384054,
 'mape_mc4': 1.953615610380001,
 'mape_rpg': 3.701434194421058}

In [169]:

def _safe_publish_record(pub: PIPublisher, record: dict, ts):
    tz = pub.cfg.tz
    ts_local = pd.Timestamp(ts)
    ts_local = ts_local.tz_localize(tz) if ts_local.tz is None else ts_local.tz_convert(tz)

    # clamp future just in case
    now_pi = pd.Timestamp.now(tz=tz).floor("T")
    if ts_local > now_pi:
        ts_local = now_pi - pd.Timedelta(seconds=1)

    for key, tag in TAG_MAP.items():
        if key not in record:
            continue
        val = record[key]
        if val is None or (isinstance(val, float) and np.isnan(val)):
            continue
        try:
            pt = pub._get_point(tag)
            ptype = (getattr(pt, "pointtype", "") or "").lower()

            if tag in STRING_TAGS:
                # if PI point isn't actually string, skip to avoid float coercion errors
                if ptype != "string":
                    print(f"[skip] {tag}: PI point type is {ptype}, expected 'string'")
                    continue
                v = str(val)
            else:
                # numeric path
                v = float(val)

            pt.update_value(v, ts_local.to_pydatetime(),
                            UpdateMode.NO_REPLACE, BufferMode.BUFFER_IF_POSSIBLE)
        except Exception as e:
            print(f"[fail] {tag}: {e}")


In [170]:

def last_9_or_21(now_local: pd.Timestamp) -> pd.Timestamp:
    # returns the last 09:00 or 21:00 <= now (tz-naive)
    now_local = pd.Timestamp(now_local).tz_convert('Asia/Seoul') if now_local.tzinfo else now_local.tz_localize('Asia/Seoul')
    last = ((now_local - pd.Timedelta(hours=9)).floor('12H') + pd.Timedelta(hours=9))
    return last.tz_localize(None)


# ===== EFOM → PI push helpers =====
TAG_MAP = {
    "rcot1": "M10_EFOM_RCOT1",
    "rcot2": "M10_EFOM_RCOT2",
    "rcot3": "M10_EFOM_RCOT3",
    "rcot4_nap": "M10_EFOM_RCOT4_NAP",
    "rcot4_gas": "M10_EFOM_RCOT4_GAS",
    "rcot5_nap": "M10_EFOM_RCOT5_NAP",
    "rcot5_gas": "M10_EFOM_RCOT5_GAS",
    "rcot6_nap": "M10_EFOM_RCOT6_NAP",
    "rcot6_gas": "M10_EFOM_RCOT6_GAS",
    "eth_prod": "M10_EFOM_ETH_PROD",
    "prop_prod": "M10_EFOM_PROP_PROD",
    "mc4_prod": "M10_EFOM_MC4_PROD",
    "rpg_prod": "M10_EFOM_RPG_PROD",
    "margin_hourly": "M10_EFOM_MARGIN_HOURLY",
    "performance": "M10_EFOM_PERFORMANCE",
    # "timestamp_str": "M10_EFOM_TIMESTAMP",
    "mape_ethy": "M10_EFOM_MAPE_ETHY",
    "mape_prop": "M10_EFOM_MAPE_PROP",
    "mape_mc4": "M10_EFOM_MAPE_MC4",
    "mape_rpg": "M10_EFOM_MAPE_RPG",
}
STRING_TAGS = {"M10_EFOM_TIMESTAMP"}  # tags we *must* write as strings


def _extract_mape_for_push(metrics: pd.DataFrame) -> dict:
    """Return {'mape_ethy': float|None, 'mape_prop': ..., 'mape_mc4': ..., 'mape_rpg': ...} from metrics df."""
    if metrics is None or metrics.empty:
        return {}
    def get(tgt):
        try:
            v =metrics[metrics['target'] == tgt]['mape_pct'].mean()
            return float(v) if pd.notna(v) else None
        except Exception:
            return None
    return {
        'mape_ethy': get('Ethylene_prod_t+1'),
        'mape_prop': get('Propylene_prod_t+1'),
        'mape_mc4':  get('MixedC4_prod_t+1'),
        'mape_rpg':  get('RPG_prod_t+1'),
    }

def _latest_recs_from_outdir(out_dir: Path):
    # try usual names, then wildcard
    cand = [
        out_dir /"online" / "rcot_recommendations_sim.csv",
        out_dir /"online" / "rcot_recommendations.csv",

    ]
    if not any(p.exists() for p in cand):
        # fallback to first matching file if any
        wild = sorted(out_dir.glob("rcot_recommendations*.csv"))
        if wild:
            cand.append(wild[-1])
    rec_path = next((p for p in cand if p.exists()), None)
    if not rec_path:
        return None, {}, {}, {}
    recs = pd.read_csv(rec_path, parse_dates=["timestamp"]).sort_values("timestamp")
    last = recs.iloc[-1]
    ts = pd.Timestamp(last["timestamp"])
    # rcots
    rcots = {}
    for c in recs.columns:
        if c.startswith("rcot_opt_") and pd.notna(last[c]):
            rcots[c.replace("rcot_opt_", "")] = float(last[c])
    # products (current baseline; switch to *_opt_tph if you want optimal)
    prods = {}
    for p in ["Ethylene","Propylene","MixedC4","RPG"]:
        col = f"{p}_current_tph"
        if col in recs.columns and pd.notna(last[col]):
            prods[p] = float(last[col])
    # margin/performance if you stored them (else zeros)
    extras = dict(
        margin_hourly=float(last.get("improvement_per_h", 0.0)) if pd.notna(last.get("improvement_per_h", np.nan)) else 0.0,
        performance=0.0,  # plug your own score if you have it
    )
    return ts, rcots, prods, extras

def _fallback_from_frames(X_12h: pd.DataFrame, Y_12h: pd.DataFrame):
    last = X_12h.dropna(how="all").iloc[-1]
    ts = last.name
    rcots = {}
    for k in ["RCOT_chamber1","RCOT_chamber2","RCOT_chamber3",
              "RCOT_naphtha_chamber4","RCOT_gas_chamber4",
              "RCOT_naphtha_chamber5","RCOT_gas_chamber5",
              "RCOT_naphtha_chamber6","RCOT_gas_chamber6"]:
        if k in last and pd.notna(last[k]): rcots[k] = float(last[k])
    prods = {}
    if not Y_12h.empty and ts in Y_12h.index:
        for p in ["Ethylene","Propylene","MixedC4","RPG"]:
            col = f"{p}_prod_t+1"
            if col in Y_12h.columns and pd.notna(Y_12h.at[ts, col]):
                prods[p] = float(Y_12h.at[ts, col])
    extras = dict(margin_hourly=0.0, performance=0.0)
    return ts, rcots, prods, extras

def _build_push_record(out_dir: Path, X_12h: pd.DataFrame, Y_12h: pd.DataFrame, mape: dict | None = None):
    ts, rcots, prods, extras = _latest_recs_from_outdir(out_dir)
    if ts is None:
        ts, rcots, prods, extras = _fallback_from_frames(X_12h, Y_12h)

    def pick(d, *keys):
        for k in keys:
            if k in d: return d[k]
        return None

    rec = {
        "rcot1":     pick(rcots, "RCOT_chamber1"),
        "rcot2":     pick(rcots, "RCOT_chamber2"),
        "rcot3":     pick(rcots, "RCOT_chamber3"),
        "rcot4_nap": pick(rcots, "RCOT_naphtha_chamber4"),
        "rcot4_gas": pick(rcots, "RCOT_gas_chamber4"),
        "rcot5_nap": pick(rcots, "RCOT_naphtha_chamber5"),
        "rcot5_gas": pick(rcots, "RCOT_gas_chamber5"),
        "rcot6_nap": pick(rcots, "RCOT_naphtha_chamber6"),
        "rcot6_gas": pick(rcots, "RCOT_gas_chamber6"),
        "eth_prod":  prods.get("Ethylene"),
        "prop_prod": prods.get("Propylene"),
        "mc4_prod":  prods.get("MixedC4"),
        "rpg_prod":  prods.get("RPG"),
        "margin_hourly": extras.get("margin_hourly", 0.0),
        "performance":   extras.get("performance", 0.0),
        "timestamp_str": pd.Timestamp(ts).strftime("%Y-%m-%d %H:%M:%S"),
    }
    if mape:
        rec.update({k: (None if v is None else float(v)) for k, v in mape.items()})

    rec = {k: v for k, v in rec.items() if v is not None}
    return ts, rec

In [171]:
# ts_push.astype(str) is this correct? 
str(ts_push)

'2025-09-22 09:00:00'

In [172]:
# === gate helpers ===
GATE_TAG = "M10_EFOM_CYCLE_SUCCESS"  # handshake gate on the PI side

def _write_scalar(pub, tag: str, value: float, when_ts=None):
    """Write a numeric scalar to PI at 'when_ts' (or now) with clamped time."""
    tz = pub.cfg.tz
    ts_local = pd.Timestamp.now(tz=tz).floor("T") if when_ts is None else pd.Timestamp(when_ts)
    ts_local = ts_local.tz_localize(tz) if ts_local.tz is None else ts_local.tz_convert(tz)
    # clamp to <= now (PI rejects future)
    now_pi = pd.Timestamp.now(tz=tz).floor("T")
    if ts_local > now_pi:
        ts_local = now_pi - pd.Timedelta(seconds=1)
    pt = pub._get_point(tag)
    v = float(value)
    pt.update_value(v, ts_local.to_pydatetime(), UpdateMode.NO_REPLACE, BufferMode.BUFFER_IF_POSSIBLE)

def publish_with_gate(out_dir: Path, X_12h, Y_12h, mape=None, done_value: float = 1.0):
    """Gate=0 → payload → Gate=done_value (usually 1.0)."""
    ts_push, rec = _build_push_record(out_dir, X_12h, Y_12h, mape=mape)

    with PIPublisher(PIServerConfig(server="PISRV_ALIAS_OR_REGISTERED_NAME", tz="Asia/Seoul")) as pub:
        # 1) pre-gate → 0 at NOW
        try:
            _write_scalar(pub, GATE_TAG, 0.0, when_ts=None)   # write at now
            print(f"[GATE] {GATE_TAG}=0")
        except Exception as e:
            print(f"[WARN] gate pre-write failed: {e}")

        # 2) payload at ts_push
        _safe_publish_record(pub, rec, ts_push)

        # 3) post-gate → done (default 1.0), at NOW
        try:
            _write_scalar(pub, GATE_TAG, done_value, when_ts=None)
            print(f"[GATE] {GATE_TAG}={done_value}")
        except Exception as e:
            print(f"[WARN] gate post-write failed: {e}")


In [173]:
# from PIconnect.PIConsts import UpdateMode, BufferMode

# # save
# # OUT_DIR.mkdir(parents=True, exist_ok=True)
# # metrics.to_excel(OUT_DIR / "metrics.xlsx")
# # preds.to_csv(OUT_DIR / "preds_eval.csv")
# # Build MAPE dict from the just-computed metrics
# mape_for_push = _extract_mape_for_push(metrics)

# # Push (only in online if you prefer, or in all modes)
# try:
#     ts_push, rec = _build_push_record(OUT_DIR, X_12h, Y_12h, mape=mape_for_push)
#     ts_push = str(ts_push)
#     with PIPublisher(PIServerConfig(server="172.17.21.117", tz="Asia/Seoul")) as pub:
#         _safe_publish_record(pub, rec, ts_push)
#     print(f"[OK] Published EFOM setpoints & MAPEs at {ts_push}")
# except Exception as e:
#     print(f"[WARN] PI publish skipped: {e}")


In [176]:
rec

{'rcot1': 844.8678791249838,
 'rcot3': 844.8229165601236,
 'rcot4_gas': 884.7312444051107,
 'rcot5_gas': 884.7832336425782,
 'eth_prod': 80.2329942262229,
 'prop_prod': 39.66314831375817,
 'mc4_prod': 21.572257048274647,
 'rpg_prod': 40.95550052808159,
 'margin_hourly': 50234.41828429593,
 'performance': 0.0,
 'timestamp_str': '2025-09-22 09:00:00',
 'mape_ethy': 1.587290104251739,
 'mape_prop': 2.220742953384054,
 'mape_mc4': 1.953615610380001,
 'mape_rpg': 3.701434194421058}

In [178]:
# pick the correct OUT folder for this mode (usually OUT_DIR / "online")
MODE_DIR = OUT_DIR / ("online" if MODE == "online" else MODE)

# mape_for_push from your metrics (or {})
mape_for_push = _extract_mape_for_push(metrics) if 'metrics' in locals() else {}

try:
    publish_with_gate(MODE_DIR, X_12h, Y_12h, mape=mape_for_push, done_value=0.0)
    print('success')
except Exception as e:
    print(f"[WARN] gated publish skipped: {e}")


c:\ProgramData\miniconda3\envs\efom\Lib\site-packages\PIconnect\PI.py:85: UserWarning: Server "PISRV_ALIAS_OR_REGISTERED_NAME" not found, using the default server.
  warnings.warn(


[GATE] M10_EFOM_CYCLE_SUCCESS=0
[GATE] M10_EFOM_CYCLE_SUCCESS=0.0
success


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_7400\1540697280.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ts_local = pd.Timestamp.now(tz=tz).floor("T") if when_ts is None else pd.Timestamp(when_ts)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_7400\1540697280.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  now_pi = pd.Timestamp.now(tz=tz).floor("T")


In [145]:
pd.read_csv('C:/Administrator/Documents/prod_out/jupyter_v3/online/rcot_recommendations_sim.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Administrator/Documents/prod_out/jupyter_v3/online/rcot_recommendations_sim.csv'

In [148]:
pd.read_csv('C:\Administrator\Documents\prod_out\jupyter_v3\online\rcot_recommendations_sim.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Administrator\\Documents\\prod_out\\jupyter_v3\\online\rcot_recommendations_sim.csv'

In [177]:
rec

{'rcot1': 844.8678791249838,
 'rcot3': 844.8229165601236,
 'rcot4_gas': 884.7312444051107,
 'rcot5_gas': 884.7832336425782,
 'eth_prod': 80.2329942262229,
 'prop_prod': 39.66314831375817,
 'mc4_prod': 21.572257048274647,
 'rpg_prod': 40.95550052808159,
 'margin_hourly': 50234.41828429593,
 'performance': 0.0,
 'timestamp_str': '2025-09-22 09:00:00',
 'mape_ethy': 1.587290104251739,
 'mape_prop': 2.220742953384054,
 'mape_mc4': 1.953615610380001,
 'mape_rpg': 3.701434194421058}